In [64]:
import numpy
import scipy.io
import math
import geneNewData

def get_mean(sample_image):
    flattened_sample_image=sample_image.flatten()
    return sum(flattened_sample_image)/len(flattened_sample_image)
def get_std(sample_image):
    mean=get_mean(sample_image)
    flattened_sample_image=sample_image.flatten()    
    return math.sqrt(sum((flattened_sample_image-mean)**2)/len(flattened_sample_image) )

#since we have assumed that these two features are independent, and that each image is drawn from a normal distribution. 
def likelihood_fun(x, mean,var):
    return (1/math.sqrt(2*math.pi*var) )*math.exp((-(x-mean)**2)/(2*var))

def posterior_prob(params,example):
    #class_prob_0=0.5
    #class_prob_1=0.5
    posterior_0= math.log(likelihood_fun( example[0],params[0][0],params[0][1]) )+ math.log(likelihood_fun( example[1],params[0][2],params[0][3])) #+math.log(.5)
    posterior_1= math.log(likelihood_fun( example[0],params[1][0],params[1][1]) )+ math.log(likelihood_fun( example[1],params[1][2],params[1][3])) #+math.log(.5)
    predict= 0 if posterior_0 > posterior_1 else 1
    return predict

def main():
    myID='4427'
    geneNewData.geneData(myID)
    Numpyfile0 = scipy.io.loadmat('digit0_stu_train'+myID+'.mat')
    Numpyfile1 = scipy.io.loadmat('digit1_stu_train'+myID+'.mat')
    Numpyfile2 = scipy.io.loadmat('digit0_testset'+'.mat')
    Numpyfile3 = scipy.io.loadmat('digit1_testset'+'.mat')
    train0 = Numpyfile0.get('target_img')
    train1 = Numpyfile1.get('target_img')
    test0 = Numpyfile2.get('target_img')
    test1 = Numpyfile3.get('target_img')
    print([len(train0),len(train1),len(test0),len(test1)])
    print('Your trainset and testset are generated successfully!')
    #    pass
    #--------task 1
    #avg_brightness_0=[numpy.mean(image) for image in train0] 
    avg_brightness_0= list(map(get_mean,train0))
    #std_brightness_0=[numpy.std(image) for image in train0] 
    std_brightness_0= list(map(get_std,train0))
    #avg_brightness_1=[numpy.mean(image) for image in train1] 
    avg_brightness_1=list(map(get_mean,train1))
    #std_brightness_1=[numpy.std(image) for image in train1] 
    std_brightness_1= list(map(get_std,train1))

    #-------task 2
    #mean_avg_brightness_0=numpy.mean(avg_brightness_0)
    mean_avg_brightness_0=sum(avg_brightness_0)/len(avg_brightness_0)
    #var_avg_brightness_0=numpy.var(avg_brightness_0)
    var_avg_brightness_0= sum((x-mean_avg_brightness_0)**2 for x in avg_brightness_0)/len(avg_brightness_0)
    #mean_std_brightness_0=numpy.mean(std_brightness_0)
    mean_std_brightness_0=sum(std_brightness_0)/len(std_brightness_0)
    #var_std_brightness_0= numpy.var(std_brightness_0)
    var_std_brightness_0=sum((x-mean_std_brightness_0)**2 for x in std_brightness_0)/len(std_brightness_0)

    #mean_avg_brightness_1=numpy.mean(avg_brightness_1)
    mean_avg_brightness_1=sum(avg_brightness_1)/len(avg_brightness_1)
    #var_avg_brightness_1=numpy.var(avg_brightness_1)
    var_avg_brightness_1= sum((x-mean_avg_brightness_1)**2 for x in avg_brightness_1)/len(avg_brightness_1)
    #mean_std_brightness_1=numpy.mean(std_brightness_1)
    mean_std_brightness_1=sum(std_brightness_1)/len(std_brightness_1)
    #var_std_brightness_1= numpy.var(std_brightness_1)
    var_std_brightness_1=sum((x-mean_std_brightness_1)**2 for x in std_brightness_1)/len(std_brightness_1)

    print(mean_avg_brightness_0,var_avg_brightness_0,
          mean_std_brightness_0,var_std_brightness_0,
          mean_avg_brightness_1,var_avg_brightness_1
         ,mean_std_brightness_1,var_std_brightness_1)

    #----------task 3 and 4
    data_train_0=[[avg_brightness_0[i],std_brightness_0[i]] for i in range(len(avg_brightness_0))]
    data_train_1=[[avg_brightness_1[i],std_brightness_1[i]] for i in range(len(avg_brightness_1))]
    params=[
        [mean_avg_brightness_0,var_avg_brightness_0,mean_std_brightness_0,var_std_brightness_0],

        [mean_avg_brightness_1,var_avg_brightness_1,mean_std_brightness_1,var_std_brightness_1]
    ]
    #training accuracry
    pred_0=[]
    pred_1=[]
    for i,j in enumerate(data_train_0):
        pred_0.append(posterior_prob(params,data_train_0[i]))
        pred_1.append(posterior_prob(params,data_train_1[i]))
    test_level=pred_0+pred_1
    train_level_0=list(0 * numpy.ones(len(data_train_0)))
    train_level_1=list(1* numpy.ones(len(data_train_1)))
    train_level=train_level_0+train_level_1
    accuracy= sum(1 for x,y in zip(train_level,test_level) if x==y)/len(train_level)
    print("training accuracy",accuracy)

    #testing
    '''
    avg_brightness_0=[numpy.mean(image) for image in test0] 
    std_brightness_0=[numpy.std(image) for image in test0]
    avg_brightness_1=[numpy.mean(image) for image in test1] 
    std_brightness_1=[numpy.std(image) for image in test1] 
    '''
    avg_brightness_0= list(map(get_mean,test0))
    std_brightness_0= list(map(get_std,test0))
    avg_brightness_1= list(map(get_mean,test1))
    std_brightness_1= list(map(get_std,test1))

    data_test_0=[[avg_brightness_0[i],std_brightness_0[i]] for i in range(len(avg_brightness_0))]
    data_test_1=[[avg_brightness_1[i],std_brightness_1[i]] for i in range(len(avg_brightness_1))]

    test_pred_0=[]
    test_pred_1=[]
    for i,j in enumerate(data_test_0):
        test_pred_0.append(posterior_prob(params,data_test_0[i]))
    for i,j in enumerate(data_test_1):
        test_pred_1.append(posterior_prob(params,data_test_1[i]))
    test_pred=test_pred_0+test_pred_1
    test_level_0=list(0 * numpy.ones(len(data_test_0)))
    test_level_1=list(1* numpy.ones(len(data_test_1)))
    Accuracy_for_digit0testset = sum(1 for x,y in zip(test_level_0,test_pred_0) if x==y)/len(test_level_0)
    print("class 0 accuracy",Accuracy_for_digit0testset)
    Accuracy_for_digit1testset= sum(1 for x,y in zip(test_level_1,test_pred_1) if x==y)/len(test_level_1)
    print("class 1 acccuracy",Accuracy_for_digit1testset)

if __name__ == '__main__':
    main()

[5000, 5000, 980, 1135]
Your trainset and testset are generated successfully!
44.2146428571 115.550258903 87.43059533674777 101.4688032971233 19.4110196429 32.7003438061 61.386556126668715 84.8037115156687
training accuracy 0.9237
class 0 accuracy 0.9163265306122449
class 1 acccuracy 0.9233480176211454


# check1:
# using mean and std from numpy library

In [21]:

# 44.2146428571 115.550258903 87.4305953367 101.468803297 19.4110196429 32.7003438061 61.3865561267 84.8037115157
#0.9163265306122449
#0.9233480176211454

# check2:
# verifying with  sklearn GaussianNB standard librarry


In [36]:
data=[data_train_0.append(item) for item in data_train_1]

In [65]:
data=data_train_0
train_level
from sklearn.naive_bayes import GaussianNB
clf2 = GaussianNB()
clf2.class_prior_ = [0.5, 0.5]
clf2.fit(numpy.array(data),numpy.array(train_level))
re1=clf2.predict(numpy.array(data_test_0))
res1=list(re1)
Accuracy_for_digit0testset= sum(1 for x,y in zip(re1,test_level_0) if x==y)/len(test_level_0)
print(Accuracy_for_digit0testset)
re2=clf2.predict(numpy.array(data_test_1))
res2=list(re2)
Accuracy_for_digit1testset= sum(1 for x,y in zip(re2,test_level_1) if x==y)/len(test_level_1)
print(Accuracy_for_digit1testset)

0.9163265306122449
0.9233480176211454


In [31]:
clf2.sigma_

array([[ 115.55025917,  101.46880356],
       [  32.70034407,   84.80371178]])

In [32]:
clf2.theta_

array([[ 44.21464286,  87.43059534],
       [ 19.41101964,  61.38655613]])